https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        # チャネル数1の画像を入力とするとき、
        # 6枚の畳込みフィルターを定義する。
        # フィルタのサイズは5である。
        # このとき、出力は
        # チャネル数6チャネル=フィルタ数になる
        # 画素数は、入力サイズがLならばL-(フィルタサイズ-1)=L-5+1=L-4になる
        self.conv1 = nn.Conv2d(1, 6, 5)
        # チャネル数6入力として、
        # 16枚の畳込みフィルターを定義する。
        # フィルタのサイズは5である。
        # このとき、出力は
        # 16チャネル=フィルタ数になる
        # 画素数は、入力サイズがLならばL-(フィルタサイズ-1)=L-5+1=L-4になる
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        # 16*5*5 のうち、16は単にチャネル数を示す。
        # 2つの5はそれぞれ実際の画素数を表す。
        # fc1に対する入力サイズは、もとの画像サイズをL=32とすると、
        # それぞれ2回の畳込とpoolingによって5*5になっているので、
        # 総計して合計要素数は16*5*5となる。
        # 120は出力数
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # 入力数120, 出力数84
        self.fc2 = nn.Linear(120, 84)
        # 入力数84, 出力数10
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        # (2,2) サイズのウィンドウで pooling する
        # (L, L) -> (L/2, L/2) になる
        # 入力のxがL=32と仮定すると、
        # conv1 によって32 -> 32-4 = 28 になっているので、
        # pooling で 28 -> 14 になる
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        # (2,2) サイズのwウィンドウで pooling する
        # 画像サイズが正方形なら、
        # (2,2) とタプルで指定せずに、単に 2 でよいということのようだ。
        # 入力のxがL=14 と仮定すると、
        # conv2 によって 14 -> 14-4 = 10 になっているので、
        # pooling で 10 -> 10/2 になる
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # affine変換のために1次元配列化する
        x = x.view(-1, self.num_flat_features(x))
        # fc1でアフィン変換したあと活性化関数をかける
        x = F.relu(self.fc1(x))
        # fc2でアフィン変換したあと活性化関数をかける
        x = F.relu(self.fc2(x))
        # fc3でアフィン変換する
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [30]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [31]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0433, -0.0334, -0.0871,  0.0420, -0.0992, -0.0831,  0.0282,  0.1429,
          0.0222, -0.0059]], grad_fn=<AddmmBackward>)


In [32]:
output = net(input)
# ここでは教師データをダミーで作っておく
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
# Loss function として MSE(Mean Square Error: 二乗誤差）を使う
criterion = nn.MSELoss()

In [33]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update